### 이번 프로젝트 목표
---  

**공통**  

1. BERT pretrained model을 활용한 KorQuAD 모델을 정상적으로 학습 시키기  
    - KorQuAD 모델의 validation accuracy가 안정적으로 증가시킴

2. KorQuAD Inference 결과가 원래의 정답과 비교하여 유사하게 나오는 것을 확인하기
    - 평가셋에 대해 모델 추론 결과와 실제 정답의 유사성이 확인

3. pretrained model 활용이 효과적임을 실험을 통해 확인하기  
    - pretrained model을 사용하지 않았을 때 대비 학습경과의 차이를 시각화를 통해 확인

---
**개인**
1. BERT 모델 이해하고 코드 typing 진행
2. 하이퍼 파라미터별 모델 성능 비교

## STEP 0. 라이브러리 불러오기
---

In [ ]:
from importlib.metadata import version
import tensorflow
import sentencepiece
import tqdm